<a href="https://colab.research.google.com/github/mahmoud-ekhani/LLM_tutorials/blob/main/Fine_tuning_RoBERTa_Unhealthy_Comment_Corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The plan for fine-tuning RoBERTa with the UCC

### 1. Set up
- Install transformers and pytorch lightning and import required packages.
- Loads UCC data and inspect it.

### 2. Create a dataset

- Load data into pytorch dataset

### 3. Build the model
 - Create a new pytorch (lightning) model with the Hugging Face RoBERTa model and a multi-label classification head.

### 4. Test model performance
- Test model performance using the ROC AUC and same test-set used in the original paper.

In [ ]:
!nvidia-smi

# Imports

In [ ]:
%pip install transformers
%pip install pytorch-lightning
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
%pip install tensorboard


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train_path = 'data/ucc/train.csv'
val_path = 'data/ucc/val.csv'

In [ ]:
train_data = pd.read_csv(train_path)

In [ ]:
train_data.head(5)

# inspect data

In [ ]:
train_data['unhealthy'] = np.where(train_data['healthy'] == 1, 0, 1)

attributes = ['antagonize', 'condescending','dismissive','generalisation',
              'generalisation_unfair','hostile','sarcastic','unhealthy']

train_data[attributes].sum().plot.bar()

# Dataset

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class UCC_Dataset(Dataset):

  def __init__(self, data_path, tokenizer, attributes, max_token_len: int = 128, sample = 5000):
    self.data_path = data_path
    self.tokenizer = tokenizer
    self.attributes = attributes
    self.max_token_len = max_token_len
    self.sample = sample
    self._prepare_data()


  def _prepare_data(self):
    data = pd.read_csv(self.data_path)
    data['unhealthy'] = np.where(data['healthy'] == 1, 0, 1)
    if self.sample is not None:
      unhealthy = data.loc[data[attributes].sum(axis=1) > 0]
      clean = data.loc[data[attributes].sum(axis=1) == 0]
      self.data = pd.concat([unhealthy, clean.sample(self.sample, random_state=42)])
    else:
      self.data = data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    item = self.data.iloc[index]
    comment = str(item.comment)
    attributes = torch.FloatTensor(self.data.iloc[index, self.data.columns.get_indexer(self.attributes)])
    tokens = self.tokenizer.encode_plus(comment,
                                        add_special_tokens=True,
                                        return_tensors='pt',
                                        truncation=True,
                                        padding='max_length',
                                        max_length=self.max_token_len,
                                        return_attention_mask = True)
    return {'input_ids': tokens.input_ids.flatten(), 'attention_mask': tokens.attention_mask.flatten(), 'labels': attributes}


In [ ]:
import torch
from transformers import AutoTokenizer
model_name = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
ucc_ds = UCC_Dataset(train_path, tokenizer, attributes=attributes)
ucc_ds_val = UCC_Dataset(val_path, tokenizer, attributes=attributes, sample=None)

In [ ]:
ucc_ds.__getitem__(0)

In [ ]:
ucc_ds.__getitem__(0)['labels'].shape, ucc_ds.__getitem__(0)['input_ids'].shape, ucc_ds.__getitem__(0)['attention_mask'].shape

In [ ]:
len(ucc_ds)

In [ ]:
len(ucc_ds_val)

# Data module

In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader

In [ ]:
class UCC_Data_Module(pl.LightningDataModule):

  def __init__(self, train_path, val_path, attributes, batch_size: int = 16, max_token_length: int = 128,  model_name='roberta-base'):
    super().__init__()
    self.train_path = train_path
    self.val_path = val_path
    self.attributes = attributes
    self.batch_size = batch_size
    self.max_token_length = max_token_length
    self.model_name = model_name
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)

  def setup(self, stage = None):
    if stage in (None, "fit"):
      self.train_dataset = UCC_Dataset(self.train_path, attributes=self.attributes, tokenizer=self.tokenizer)
      self.val_dataset = UCC_Dataset(self.val_path, attributes=self.attributes, tokenizer=self.tokenizer, sample=None)
    if stage == 'predict':
      self.val_dataset = UCC_Dataset(self.val_path, attributes=self.attributes, tokenizer=self.tokenizer, sample=None)

  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size = self.batch_size, num_workers=4, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size = self.batch_size, num_workers=4, shuffle=False)

  def predict_dataloader(self):
    return DataLoader(self.val_dataset, batch_size = self.batch_size, num_workers=4, shuffle=False)


In [ ]:
ucc_data_module = UCC_Data_Module(train_path, val_path, attributes=attributes)

In [ ]:
ucc_data_module.setup()

In [ ]:
ucc_data_module.train_dataloader()

In [ ]:
# Number of batches (9960 // 16 = 623)
len(ucc_data_module.train_dataloader())

# Model

In [ ]:
from transformers import AutoModel, AdamW, get_cosine_schedule_with_warmup
import torch.nn as nn
import math
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F

In [ ]:
class UCC_Comment_Classifier(pl.LightningModule):

    def __init__(self, config: dict):
        super().__init__()
        self.config = config
        self.pretrained_model = AutoModel.from_pretrained(config['model_name'], return_dict=True)
        self.hidden = nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size)
        self.classifier = nn.Linear(self.pretrained_model.config.hidden_size, self.config['n_labels'])
        torch.nn.init.xavier_uniform_(self.hidden.weight)
        torch.nn.init.xavier_uniform_(self.classifier.weight)
        self.loss_fun = nn.BCEWithLogitsLoss(reduction='mean') # Mean reduces this to a single number, combines logits, sigmoids, and cross-entropy
        self.dropout = nn.Dropout() # randomly turns on and off some nodes during each round of training - prevents over-fitting (regularization)

    def forward(self, input_ids, attention_mask, labels=None):
        # roberta model
        output = self.pretrained_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output =  torch.mean(output.last_hidden_state, 1) # mean over the first dimension (tokens)
        # classification layers
        hidden_output = self.hidden(pooled_output)
        hidden_output = self.dropout(hidden_output)
        hidden_output = F.relu(hidden_output)
        logits = self.classifier(hidden_output)
        # calculate the loss
        loss = 0
        if labels is not None:
            loss = self.loss_fun(logits.view(-1, self.config['n_labels']), labels.view(-1, self.config['n_labels']))
        return loss, logits

    def training_step(self, batch, batch_index):
        loss, logits = self(**batch)
        self.log("train loss,", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": logits, "labels": batch['labels']}

    def validation_step(self, batch, batch_index):
        loss, logits = self(**batch)
        self.log("validation loss,", loss, prog_bar=True, logger=True)
        return {"val_loss": loss, "predictions": logits, "labels": batch['labels']}

    def predict_step(self, batch, batch_index):
        _, logits = self(**batch)
        return logits

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.config['lr'], weight_decay=self.config['weight_decay'])
        total_steps = self.config['train_size'] // self.config['batch_size']
        warmup_steps = math.floor(total_steps * self.config['warmup'])
        scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
        return [optimizer], [scheduler]


In [ ]:
config = {
    'model_name': 'distilroberta-base',
    'n_labels': len(attributes),
    'batch_size': 128,
    'lr': 1.5e-6,
    'warmup': 0.2,
    'train_size': len(ucc_data_module.train_dataloader()),
    'weight_decay': 0.001,
    'n_epochs': 1
}

model = UCC_Comment_Classifier(config)

In [ ]:
idx=0
input_ids = ucc_ds.__getitem__(idx)['input_ids']
attention_mask = ucc_ds.__getitem__(idx)['attention_mask']
labels = ucc_ds.__getitem__(idx)['labels']
model.cpu()
loss, output = model(input_ids.unsqueeze(dim=0), attention_mask.unsqueeze(dim=0), labels.unsqueeze(dim=0))
print(loss)
print(labels.shape, output.shape, output.detach().numpy())

 # train model

In [ ]:
# datamodule
ucc_data_module = UCC_Data_Module(train_path, val_path, attributes=attributes, batch_size=config['batch_size'])
ucc_data_module.setup()

# model
model = UCC_Comment_Classifier(config)

# trainer and fit
trainer = pl.Trainer(max_epochs=config['n_epochs'], num_sanity_val_steps=50)
trainer.fit(model, ucc_data_module)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs/

# Predict with model

In [ ]:
# method to convert list of comments into predictions for each comment
def classify_raw_comments(model, dm):
  predictions = trainer.predict(model, datamodule=dm)
  flattened_predictions = np.stack([torch.sigmoid(torch.Tensor(p)) for batch in predictions for p in batch])
  return flattened_predictions

In [ ]:
predictions = classify_raw_comments(model, ucc_data_module)

In [ ]:
val_data = pd.read_csv(val_path)
val_data['unhealthy'] = np.where(val_data['healthy'] == 1, 0, 1)
true_labels = np.array(val_data[attributes])

In [ ]:
from sklearn import metrics
plt.figure(figsize=(15, 8))
for i, attribute in enumerate(attributes):
  fpr, tpr, _ = metrics.roc_curve(
      true_labels[:,i].astype(int), predictions[:, i])
  auc = metrics.roc_auc_score(
      true_labels[:,i].astype(int), predictions[:, i])
  plt.plot(fpr, tpr, label='%s %g' % (attribute, auc))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.title('RoBERTa Trained on UCC Datatset - AUC ROC')